<a href="https://colab.research.google.com/github/HeqingZ/TESTING/blob/main/EXPERIMENT_SMALL_SWINT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#mount on google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#install detectron2
!python -m pip install pyyaml==5.1
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Created wheel for detectron2: filename=detectron2-0.6-cp37-cp37m-linux_x86_64.whl size=5240737 sha256=f0b286b4908d32c88a53fae40d39812cf5c9b5eeed563ff02ebb0cf60469bbc4
  Stored in directory: /tmp/pip-ephem-wheel-cache-_33w123c/wheels/07/dc/32/0322cb484dbefab8b9366bfedbaff5060ac7d149d69c27ca5d
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220512-py3-none-any.whl size=61288 sha256=0bf436de9c408b765d9a6570a6ffe94888621399d22c3909fe1d71e90c71c093
  Stored in directory: /root/.cache/pip/wheels/68/20/f9/a11a0dd63f4c13678b2a5ec488e48078756505c7777b75b29e
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144575 sha256=509912441ab843151e7e84a83e6167033ba7510d71d050a6889c8b55e41a7a42
  Stored in directory: /root/.cache/pip/wheels/8b/8d/53/2af8772d9aec614e3fc65e53d4a993ad73c61daa8bbd85a873
  Created wheel for fairscale: filename=fairscale-0.4.6-py3-none-any.whl size=307252 sha256=563c0efff51b47cc7babda35f67033329b8784d71d1e83c839734

In [2]:
#import some important utilities of Detectron2
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
torch:  1.7 ; cuda:  cu110
detectron2: 0.5


In [3]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
#import features for the bbox
import numpy as np
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

from detectron2.structures import BoxMode

In [50]:
"""cpu"""
# !python -m pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cpu/torch1.9/index.html
"""gpu"""    
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 -f https://download.pytorch.org/whl/torch_stable.html
!pip install detectron2==0.5 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu110/torch1.7/index.html

!pip install timm
!git clone https://github.com/emiz6413/SwinT_detectron2.git swin
!curl -OL https://github.com/xiaohu2015/SwinT_detectron2/releases/download/v1.1/swin_tiny_patch4_window7_224_d2.pth  
!curl -OL https://github.com/xiaohu2015/SwinT_detectron2/releases/download/v1.1/faster_rcnn_swint_T.pth  # pretrained

!curl -OL https://github.com/emiz6413/SwinT_detectron2/releases/download/v1.3/model_0021209.pth  # trained

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu110/torch1.7/index.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'swin' already exists and is not an empty directory.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100     9    0     9    0     0     47      0 --:--:-- --:--:-- --:--:--    46
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--

In [ ]:
dir(detectron2.modeling)

In [51]:
import os
from pathlib import Path
from ast import literal_eval
import swin #swin 改成了swint
import numpy as np
import pandas as pd
import cv2
import torch
import matplotlib.pyplot as plt
from PIL import Image

from detectron2.data import (DatasetCatalog, 
                             MetadataCatalog, 
                             build_detection_test_loader
                            )
from detectron2.data.datasets.coco import convert_to_coco_json
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.structures import BoxMode
from detectron2.config import get_cfg
from detectron2.utils.logger import setup_logger
from detectron2.engine import DefaultTrainer, default_setup, hooks
from detectron2.modeling import GeneralizedRCNNWithTTA
from detectron2.evaluation import COCOEvaluator, DatasetEvaluators, inference_on_dataset
 
from swin.swint import add_swint_config #！！！！！！！！

logger = setup_logger()

In [52]:
#define the file path to retrieve images
def SMALLFLIR(source_path, img_dir, anno_dir ): #define a function. assuming user doesnt know shit about coding and enter random stuff. related to anno dir and img dir
    img_dir = os.path.join(source_path, img_dir)
    anno_dir = os.path.join(source_path, anno_dir)
    data_dicts = []
    #define annotation properties
    with open(anno_dir) as F:
      jsonfile = json.load(F) #load annotations from json file into the dictionary
    
    annotationinfo = jsonfile["annotations"]
    imagesinfo = jsonfile["images"] #retreieve info like category ID and bbox
   
    #define img sources
     
    img_dir_list = os.listdir(img_dir)#get the list of all files and directories in the specified directory.
    # print(img_dir)
    for img_info in imagesinfo: #bbox info category id
      for images in img_dir_list:  #img_dir_list = img_dir, which is source_path/img_dir(original)
        images = "data/" + images
        if images == img_info['file_name']: #see if name matches each other, because we wanna find the filenames only for the small flir out of the whole json file

          record={}
          filename = os.path.join(source_path, img_info["file_name"])
          # print(filename)
          record["file_name"] = filename
          record["image_id"] = img_info['id']
          record["height"] = img_info['height']
          record['width'] = img_info['width']
        
          box_info = []
          
          for anno in annotationinfo:
            if anno["image_id"] == img_info['id']:
              # if (anno['category_id'] <= 3) & (anno['category_id'] > 0):
              if (anno['category_id'] == 1) & (anno['category_id'] > 0): # Predict people only
                        box = {
                        "bbox": anno['bbox'],
                        'bbox_mode': BoxMode.XYWH_ABS, # Category_id starts from 0 in detectron2
                        'category_id': int(anno['category_id']-1)  #we forcibly convert this shit into integer to be able to minus 1. just incase the stuff from the dict is integer
                        #computer counts from zero but not hyman. we have to substract one to match the computer #update Jun29: i deleted -1, bcz there is no category 0 
                        } #exactly how dictionary looks like
                        imagetesthaha = cv2.imread(filename, 0) 
                        tt = cv2.rectangle(imagetesthaha, anno["bbox"], (255,0,0),2)
                        #cv2_imshow(tt) #for printing all boxes in images one by one, will crash
                        box_info.append(box)
          record["annotations"] = box_info

          data_dicts.append(record)
    return data_dicts

      
      
    #print(type(jsonfile)) #check the  type of jsonfile. dict or list or stuff



#define dataloader for training and testing sets
#train_dataloader = dataloader(training_data, batch_size=64, shuffle=True)
#test_dataloader = dataloader(test_data, batch_size=64, shuffle=True)


In [53]:
def FLIR_dataset_wrapper(source_path, img_dir, anno_dir):
    def wrapper():
        return SMALLFLIR(source_path, img_dir, anno_dir)
    return wrapper

In [55]:
DatasetCatalog.clear() #unregister stuff

In [56]:
for d in ["train", "val"]:
    DatasetCatalog.register("SMALL_FLIR_THERMAL_" + d + "_data", lambda d=d: SMALLFLIR("/content/drive/MyDrive/SMALL_FLIR_THERMAL/train", "data", "coco.json")) #changed "/directry/stuff/" +d +"shit" into this rn
    MetadataCatalog.get("SMALL_FLIR_THERMAL_" + d + "_data").set(thing_classes=["SMALL_FLIR_THERMAL"])
SMALL_FLIR_metadata = MetadataCatalog.get("SMALL_FLIR_THERMAL_train_data")


In [57]:
convert_to_coco_json('SMALL_FLIR_THERMAL_train_data', output_file='content/drive/MyDrive/output/inference/SMALL_FLIR_Thermal_Train_coco_format.json', allow_cached=False)


[08/27 00:56:34 d2.data.datasets.coco]: Converting annotations of dataset 'SMALL_FLIR_THERMAL_train_data' to COCO format ...)
[08/27 00:56:41 d2.data.datasets.coco]: Converting dataset dicts into COCO format
[08/27 00:56:41 d2.data.datasets.coco]: Conversion finished, #images: 105, #annotations: 70
[08/27 00:56:41 d2.data.datasets.coco]: Caching COCO format annotations at 'content/drive/MyDrive/output/inference/SMALL_FLIR_Thermal_Train_coco_format.json' ...


In [58]:
#to varify our thing by calling the function
dataset_dicts = SMALLFLIR("/content/drive/MyDrive/SMALL_FLIR_THERMAL/train", "data", "coco.json")


In [59]:
!nvidia-smi

Sat Aug 27 00:56:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    40W / 300W |   2323MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
import torch
torch.cuda.empty_cache()

In [49]:
annotation_file = '/content/drive/MyDrive/SMALL_FLIR_THERMAL/train/coco.json'

In [77]:
import pickle
with open("data.pickle", 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    f.seek(0)

    pickle.dump(dataset_dicts, f, pickle.HIGHEST_PROTOCOL)


In [80]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
add_swint_config(cfg)
cfg.DATASETS.TRAIN = ("SMALL_FLIR_THERMAL_train_data",)
cfg.MODEL.RESNETS.DEPTH = 50 #for mvit v2 S we are setting the depth = 101. (F)
cfg.DATASETS.TEST = ('SMALL_FLIR_THERMAL_val_data',)

# cfg.Dataset = ("SMALL_FLIR_THERMAL_train_data")


cfg.merge_from_file("/content/swin/configs/SwinT/faster_rcnn_swint_T_FPN_3x.yaml")
cfg.MODEL.WEIGHTS = "/content/faster_rcnn_swint_T.pth"





cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 4  # This is the real "batch size" commonly known to deep learning people 'changed to 5'
cfg.SOLVER.BASE_LR = 0.0001 #SET ALL TO 0.0001
cfg.SOLVER.MAX_ITER = 300    # 270000 for 3x (F)
cfg.SOLVER.STEPS = []        # do not decay learning rate
# cfg.SOLVER.WARMUP_FACTOR = 1.0 / 5000
# cfg.SOLVER.WARMUP_ITERS = 5000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256   # chose 512 for 3x
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (people). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.
cfg.MODEL.MASK_ON = False
# cfg.MODEL.RESNETS.DEFORM_MODULATED = True
cfg.MODEL.BACKBONE.FREEZE_AT = 0
cfg.OUTPUT_DIR = "/content/drive/MyDrive/SwinTEXPERIMENT"





#load pretrained weight
# # load pretrained weights
# other_weights = torch.load('/content/swin_tiny_patch4_window7_224_d2.pth')['model']
# self_weight = trainer.model.state_dict()
# for name, param in self_weight.items():
#     if name in other_weights:
#         if other_weights[name].shape == param.shape:
#             self_weight[name] = other_weights[name]
#         else:
#             print(f"size mismatch at {name}")
#     else:
#         print(f"layer {name} does not exist")
# trainer.model.load_state_dict(self_weight)


os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=True)
trainer.train()

[08/27 01:09:42 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(96, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(192, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(384, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(768, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): SwinTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
        (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      )
      (pos_drop): Dropout(p=0.0, inplace

roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
  pixel_mean
  pixel_std
  proposal_generator.anchor_generator.cell_anchors.{0, 1, 2, 3, 4}


[08/27 01:09:50 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.7/dist-packages/detectron2/modeling/roi_heads/fast_rcnn.py:103: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  num_fg = fg_inds.nonzero().numel()
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[08/27 01:09:58 d2.utils.events]:  eta: 0:01:41  iter: 19  total_loss: 3.203  loss_cls: 2.115  loss_box_reg: 0.002302  loss_rpn_cls: 1.133  loss_rpn_loc: 0.08501  time: 0.3619  data_time: 0.0379  lr: 6.427e-06  max_mem: 7796M
[08/27 01:10:05 d2.utils.events]:  eta: 0:01:34  iter: 39  total_loss: 0.9516  loss_cls: 0.1213  loss_box_reg: 0.00112  loss_rpn_cls: 0.8259  loss_rpn_loc: 0.05529  time: 0.3638  data_time: 0.0269  lr: 1.3087e-05  max_mem: 7796M
[08/27 01:10:12 d2.utils.events]:  eta: 0:01:26  iter: 59  total_loss: 0.8284  loss_cls: 0.04605  loss_box_reg: 0.0002933  loss_rpn_cls: 0.7256  loss_rpn_loc: 0.0487  time: 0.3598  data_time: 0.0287  lr: 1.9747e-05  max_mem: 7796M
[08/27 01:10:20 d2.utils.events]:  eta: 0:01:19  iter: 79  total_loss: 0.6338  loss_cls: 0.0569  loss_box_reg: 0.0001895  loss_rpn_cls: 0.53  loss_rpn_loc: 0.04692  time: 0.3709  data_time: 0.0321  lr: 2.6407e-05  max_mem: 7796M
[08/27 01:10:28 d2.utils.events]:  eta: 0:01:12  iter: 99  total_loss: 0.365  loss_cl

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

In [ ]:
from detectron2.utils.visualizer import ColorMode

#Use the final weights generated after successful training for inference  
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # ALL THRESHOLD = 0.5
#Pass the validation dataset
cfg.DATASETS.TEST = ("boardetect_val", )

predictor = DefaultPredictor(cfg)

In [ ]:
#see if this shit is wroking aka run instance on test images
from detectron2.utils.visualizer import ColorMode
dataset_dicts = SMALLFLIR("/content/drive/MyDrive/FLIR_THERMAL/val/", "data", "coco.json")
for d in random.sample(dataset_dicts, 5):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata=FLIR_metadata, 
                   scale=0.5, 
                      # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5 #ALL THRESHOLD = 0.5 
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("FLIR_THERMAL_train_data", cfg, False, output_dir="content/output/")
val_loader = build_detection_test_loader(cfg, "FLIR_THERMAL_val_data")
inference_on_dataset(trainer.model, val_loader, evaluator)